In [ ]:
import matplotlib
%matplotlib inline
matplotlib.rcParams['figure.figsize']=(15,8)


# Naiwny klasyfikator Bayesa

Z klasyfikatorem tym zapoznamy się próbując klasyfikować gatunki irysów. Jest to klasyczny już problem, często wykorzystywany przy porównywaniu różnych technik klasyfikacji. Więcej o pochodzeniu tych danych i problemie można przeczytać tu [https://en.wikipedia.org/wiki/Iris_flower_data_set]

Kod napiszemy w oparciu o implementacje klasyfikatora Bayesa z biblioteki <tt>scikit-learn</tt> [http://scikit-learn.org/stable/about.html#citing-scikit-learn]

Zaczerpniemy stamtąd:
* obiekt klasyfikatora <tt>GaussianNB</tt>
* zbiór danych
* funkcje do oceny jakości 

Na razie importujemy:

In [ ]:
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np

## Zbiór danych irys
Zapoznajemy się z danymi i wybieramy ich podzbiór do dalszej zabawy.

Ładujemy dane

In [ ]:
iris = datasets.load_iris() #https://en.wikipedia.org/wiki/Iris_flower_data_set

Zobaczmy co ten zbiór ma w środku:

In [ ]:
print(dir(iris))

Wypiszemy sobie opis danych:

In [ ]:
print(iris['DESCR'])

Wypiszmy nazwy gatunków:

In [ ]:
print(iris['target_names'])

Wypiszmy nazwy cech:

In [ ]:
print(iris['feature_names'])

Wypiszmy kodowanie gatunków. To są wyjścia, które chcielibyśmy uzyskać od wytrenowanego klasyfikatora:

In [ ]:
print(iris['target'])

Wypiszmy wartości cech. Są to dane wejściowe do klasyfkiacji. 

In [ ]:
print(iris['data'])

Zatem, np. obserwacja nr 5 ma cechy:

In [ ]:
print(iris.data[5,:])

i ma przypisaną klasę:

In [ ]:
print(iris.target[5])

Czyli jest to gatunek:

In [ ]:
print(iris.target_names[iris.target[5]])

## Ilustrowanie własności zbioru danych

Do rysowania zależniści między cechami i klasami przyda nam się własną mapę kolorów:

In [ ]:
color_map = {-1: (1, 1, 1), 0: (0, 0, .9), 1: (1, 0, 0), 2: (.5, .5, 0)}

Wytwarzamy wektor, który każdemu wierszowi w tabeli danych przypisze kolor odpowiadający gatunkowi irysa

In [ ]:
colors = [color_map[y] for y in iris.target]

### Aby przyjrzeć się zbiorowi danych warto zbadać: 
#### 1) Rozkłady cech w klasach: 
* np. histogramy. Proszę wykreślić histogramy rozkładu cech w poszczególnych klasach:

In [ ]:
plt.figure()
for f, f_name in enumerate(iris['feature_names']):
    plt.subplot(1,4,f+1)
    for k in range(3): # k - klasa
        plt.hist(iris.data[iris.target==k,f],color=color_map[k],alpha=0.3,bins=np.arange(0,8,0.2))
    plt.xlabel(str(f)+' '+ f_name)
plt.show() 

Violinplot

In [ ]:
plt.figure()
for f, f_name in enumerate(iris['feature_names']):
    plt.subplot(1,4,f+1)
    data = np.zeros((50,3))
    for k in range(3): #
        data[:,k] = iris.data[iris.target==k,f]      
    plt.violinplot(data)
    plt.xlabel(str(f)+' '+ f_name)
plt.show() 

#### Strukturę korelacji
Tu zwróćmy uwagę na orientację macierzy podawanej do funkcji `np.cov`:

In [ ]:
rho = np.cov(iris.data.T)
print(rho.shape)
plt.figure()
plt.matshow(rho)
plt.colorbar()
plt.show()

Stukturę korelacji i rozkłady można też podsumować na takim rysunku:
* w siatce prostokątnej rysujemy 
  * na przekątnej histogramy grupoweane, lub violinploty
  * pod przekątną wykres punktowy (scaterplot)

In [ ]:
plt.figure(1)
for i, name_i in enumerate(iris['feature_names']):
    for j, name_j in enumerate(iris['feature_names']):
        
        if i>j:
            plt.subplot(4,4,i*4+j+1) # i numeruje wiersze, j kolumny
            plt.scatter(iris.data[:,j],iris.data[:,i],c = colors)
            
        elif i==j:
            plt.subplot(4,4,i*4+j+1) # i numeruje wiersze, j kolumny
            for k in range(3):
                plt.hist(iris.data[iris.target==k,j],color=color_map[k],alpha=0.3)
        if j ==0:
            plt.ylabel(name_i)
        if i ==3:
            plt.xlabel(name_j)
plt.show()

## Szykujemy się do zbudowania klasyfikatora
Aby było nam łatwo ilustrować jego działanie wybieramy dwie cechy (podziały przestrzeni cech da się wtedy łatwo narysować na płaszczyźnie)

* Wybieramy cechy 1 i 2 (bo są ze sobą mało skorelowane) normalizujemy je

In [ ]:
X = np.zeros((iris.data.shape[0],2))
X[:,0] = (iris.data[:,1] - np.mean(iris.data[:,1]))/np.std(iris.data[:,1])
X[:,1] = (iris.data[:,2] - np.mean(iris.data[:,2]))/np.std(iris.data[:,2])  
plt.figure(2)
plt.scatter(X[:,0],X[:,1],c = colors)  
plt.title('Wybrane cechy po normalizacji')
plt.show()

Przyda nam się funkcja do wizualizacji dwuwymiarowych gaussów:

In [ ]:
def plot_gauss(mu,sigma,xx,yy):
    ''' Funkcja rysująca kontury funkcji gęstości prawdopodobieństwa 
       dwuwymiarowego rozkładu Gaussa'''

    XX = np.c_[xx.ravel(), yy.ravel()]    
    R = XX - mu 
    invS = np.linalg.inv(np.diag(sigma))
    z = np.zeros(len(R))
    for i in range(len(R)):
        z[i] = np.exp(-0.5*np.dot( R[i,:].T,np.dot(invS,R[i,:])))
    z.shape = xx.shape
    #plt.figure()
    plt.contourf(xx,yy,z,alpha = 0.5)
    plt.plot(mu[0],mu[1],'o')
    #plt.show()

## Tworzymy i uczymy klasyfikator
Poniżej znajduje się kod służący klasyfikacji. Proszę go uzupełnić zgodnie z komentarzami i dokumentacją:

https://scikit-learn.org/stable/modules/naive_bayes.html

https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = ... # stwórz instancję klasyfikatora Gaussian Naive Bayes 
... # dofituj parametry klasyfikatora 

#### Przedstaw rozkłady Gaussa, które zostały dopasowane do danych, skorzystaj z funkcji plot_gauss()
* średnie tych rozkładów są w gnb.theta_
* standardowe odchylenia są w gnb.sigma_

Przygotowanie siatki na której będą rysowane kontury Gaussów

In [ ]:
x_min, x_max = -3,3
y_min, y_max = -3,3
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                     np.arange(y_min, y_max, 0.1))
plt.figure(4)
for i in range(3):
    plot_gauss(...,...,xx,yy)
# dorzućmy do rysunku jeszcze oryginalne dane
plt.scatter(...,...,c = colors)
plt.title(u'Rozkłady Gaussa dopasowane do danych')
plt.show()        

## Rysowanie wyników klasyfikacji             
Przekształcamy siatkę w macierz dwukolumnową - kolumny odpowiadają cechom

In [ ]:
XX = np.c_[xx.ravel(), yy.ravel()]       

Dla każdego punktu siatki oblicz predykcję klasyfikatora  

In [ ]:
Z = ....

Te predykcje narysujemy w przestrzeni cech za pomocą funkcji  plt.contourf 

In [ ]:
plt.figure(3)
Z = Z.reshape(xx.shape)
plt.contourf(..., ..., ..., cmap=plt.cm.Paired)
# i dorzucamy oryginalne punkty
plt.scatter...
plt.title(u'Podział przestrzeni cech na klasy')
plt.show()

## Ewaluacja modelu
Teraz zajmiemy się ewaluacją dopasowanego modelu (walidacja krzyżowa). Skorzystamy z funkcji dostrczanych przez [Model evaluation](http://scikit-learn.org/stable/modules/model_evaluation.html)
* upewnij się, że dokładnie rozumiesz co zwracają te funkcje
* porównaj z definicjami z [wykładu](http://haar.zfb.fuw.edu.pl/edu/index.php/Uczenie_maszynowe_i_sztuczne_sieci_neuronowe/Wykład_Ocena_jakości_klasyfikacji) 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
...
print("classification report:")
print(classification_report...)
print("confusion matrix:")
print(confusion_matrix...)

## Porównanie  modeli
Stwórz trzy modele klasyfikatorów:
* pierwszym niech będzie korzystał z cech 1 i 2 (nasz dotychczasowy model)
* drugim niech korzysta tylko z cech 0 i 1
* trzeci niech korzysta ze wszytkich 4 cech

Porównaj miary jakości tych modeli otrzymywane za pomocą walidacji krzyżowej.

* Trzeci model można też stworzyć dla danych normalizowanych i nienormalizowanych, aby sprzwdzić czy normalizacja w tym przypadku pomaga w prawidłowej klasyfikacji.

In [ ]:
...